In [2]:
import torchvision
import torch
import gc
import warnings
import time
import numpy as np
from PIL import Image
import scipy.io as sio
import os
import pandas as pd
import seaborn as sns
import spectral

# Visualization
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from matplotlib.pyplot import cm
init_notebook_mode(connected=True)
%matplotlib inline

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
from sklearn.decomposition import IncrementalPCA

# Deep learning
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import (Input, Conv3D, BatchNormalization, Activation, Add, 
                                     MaxPooling3D, GlobalAveragePooling3D, GlobalMaxPooling3D,
                                     Flatten, Dense, Dropout, Concatenate, UpSampling3D,
                                     Reshape, Multiply, Permute, Lambda)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical

In [3]:
## Data Preprocessing Functions
def SplitTr_Te(HSI, GT, TeRatio, randomState=345):
    """Split data into training and testing sets"""
    Tr, Te, TrC, TeC = train_test_split(HSI, GT, test_size=TeRatio, random_state=randomState, stratify=GT)
    return Tr, Te, TrC, TeC

def DL_Method(HSI, numComponents = 75):
    """Dimensionality reduction using Incremental PCA"""
    RHSI = np.reshape(HSI, (-1, HSI.shape[2]))
    n_batches = 256
    inc_pca = IncrementalPCA(n_components=numComponents)
    for X_batch in np.array_split(RHSI, n_batches):
        inc_pca.partial_fit(X_batch)
    X_ipca = inc_pca.transform(RHSI)
    RHSI = np.reshape(X_ipca, (HSI.shape[0],HSI.shape[1], numComponents))
    return RHSI

def ZeroPad(HSI, margin=2):
    """Add zero padding to HSI data"""
    NHSI = np.zeros((HSI.shape[0] + 2 * margin, HSI.shape[1] + 2* margin, HSI.shape[2]))
    x_offset = margin
    y_offset = margin
    NHSI[x_offset:HSI.shape[0] + x_offset, y_offset:HSI.shape[1] + y_offset, :] = HSI
    return NHSI

def HSICubes(HSI, GT, WinSize=5, removeZeroLabels = True):
    """Extract spatial-spectral cubes from HSI data"""
    margin = int((WinSize - 1) / 2)
    zeroPaddedX = ZeroPad(HSI, margin=margin)
    # Split patches
    patchesData = np.zeros((HSI.shape[0] * HSI.shape[1], WinSize, WinSize, HSI.shape[2]))
    patchesLabels = np.zeros((HSI.shape[0] * HSI.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]   
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = GT[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

In [4]:
## Data Augmentation and Balancing
def augment_3d_hsi(sample, prob=0.5):
    """Augment 3D hyperspectral data"""
    augmented = sample.copy()
    
    # Random spatial flips
    if np.random.random() < prob:
        augmented = np.flip(augmented, axis=0)  # Vertical flip
    if np.random.random() < prob:
        augmented = np.flip(augmented, axis=1)  # Horizontal flip
    
    # Random rotation (multiples of 90 degrees)
    if np.random.random() < prob:
        rot_k = np.random.randint(1, 4)
        augmented = np.rot90(augmented, k=rot_k, axes=(0, 1))
    
    # Spectral noise injection
    if np.random.random() < prob:
        noise = np.random.normal(0, 0.01, size=augmented.shape)
        augmented = augmented + noise
        augmented = np.clip(augmented, 0, 1)
    
    # Spectral band shifting
    if np.random.random() < prob:
        shift = np.random.randint(-2, 3)
        augmented = np.roll(augmented, shift, axis=2)
    
    return augmented

def balance_dataset(X, y_onehot, target=1000):
    """Balance dataset by augmenting minority classes and downsampling majority classes"""
    y_labels = np.argmax(y_onehot, axis=1)
    X_balanced = []
    y_balanced = []
    
    # Calculate class distribution
    class_counts = np.bincount(y_labels)
    num_classes = len(class_counts)
    
    print(f"Original class distribution: {class_counts}")
    
    for cls in range(num_classes):
        # Extract samples of current class
        cls_mask = (y_labels == cls)
        X_cls = X[cls_mask]
        y_cls = y_onehot[cls_mask]
        current_count = len(X_cls)
        
        # Handle class balancing
        if current_count < target:
            # Need to upsample (augment)
            need = target - current_count
            print(f"Augmenting class {cls}: from {current_count} to {target}")
            
            # Generate augmented samples
            augmented_samples = []
            idx = 0
            while len(augmented_samples) < need:
                sample = X_cls[idx % current_count]
                augmented = augment_3d_hsi(sample)
                augmented_samples.append(augmented)
                idx += 1
            
            # Combine original and augmented samples
            X_balanced.extend(X_cls)
            X_balanced.extend(augmented_samples)
            y_balanced.extend(y_cls)
            y_balanced.extend([y_cls[i % current_count] for i in range(need)])
        else:
            # Downsample majority classes
            if current_count > target:
                indices = np.random.choice(current_count, target, replace=False)
                X_balanced.extend(X_cls[indices])
                y_balanced.extend(y_cls[indices])
                print(f"Downsampling class {cls}: from {current_count} to {target}")
            else:  # Exactly target count
                X_balanced.extend(X_cls)
                y_balanced.extend(y_cls)
                print(f"Class {cls}: keeping {current_count} samples")
    
    # Convert to numpy arrays
    X_balanced_np = np.array(X_balanced)
    y_balanced_np = np.array(y_balanced)
    
    # Verify sample count match
    assert len(X_balanced_np) == len(y_balanced_np), \
        f"Sample count mismatch: features {len(X_balanced_np)}, labels {len(y_balanced_np)}"
    
    # Print balanced distribution
    balanced_labels = np.argmax(y_balanced_np, axis=1)
    balanced_counts = np.bincount(balanced_labels, minlength=num_classes)
    print(f"Balanced class distribution: {balanced_counts}")
    
    return X_balanced_np, y_balanced_np

In [5]:
## Model Components
def channel_attention(input_feature, ratio=8):
    """Channel attention module"""
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    channel = input_feature.shape[channel_axis]
    
    # Shared dense layers
    def shared_dense(x):
        x = Dense(channel // ratio, activation='relu', 
                  kernel_initializer='he_normal', use_bias=True)(x)
        return Dense(channel, kernel_initializer='he_normal', use_bias=True)(x)
    
    # Global average and max pooling
    avg_pool = GlobalAveragePooling3D()(input_feature)
    avg_pool = Reshape((1, 1, channel))(avg_pool)
    avg_pool = shared_dense(avg_pool)
    
    max_pool = GlobalMaxPooling3D()(input_feature)
    max_pool = Reshape((1, 1, channel))(max_pool)
    max_pool = shared_dense(max_pool)
    
    # Attention weighting
    attention = Add()([avg_pool, max_pool])
    attention = Activation('sigmoid')(attention)
    
    if K.image_data_format() == "channels_first":
        attention = Permute((3, 1, 2))(attention)
    
    return Multiply()([input_feature, attention])


def spatial_attention(input_feature, kernel_size=7):
    """Spatial attention module"""
    if K.image_data_format() == "channels_first":
        cbam_feature = Permute((2, 3, 4, 1))(input_feature)
    else:
        cbam_feature = input_feature
    
    # Calculate channel averages and maxima
    avg_pool = Lambda(lambda x: tf.reduce_mean(x, axis=-1, keepdims=True))(cbam_feature)
    max_pool = Lambda(lambda x: tf.reduce_max(x, axis=-1, keepdims=True))(cbam_feature)
    
    # Feature fusion and convolution
    concat = Concatenate(axis=-1)([avg_pool, max_pool])
    attention = Conv3D(filters=1, kernel_size=kernel_size, strides=1, padding='same',
                       activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(concat)
    
    if K.image_data_format() == "channels_first":
        attention = Permute((4, 1, 2, 3))(attention)
    
    return Multiply()([input_feature, attention])


def cbam_block(input_feature, ratio=8):
    """CBAM attention mechanism module"""
    x = channel_attention(input_feature, ratio)
    x = spatial_attention(x)
    return x


def conv_bn_relu(filters, kernel_size, strides, input_layer, dilation_rate=1):
    """3D Convolution + Batch Normalization + ReLU activation combination layer"""
    x = Conv3D(filters, kernel_size, strides=strides, padding='same', 
               dilation_rate=dilation_rate, kernel_initializer='he_normal')(input_layer)
    x = BatchNormalization()(x)
    return Activation('relu')(x)


def mcdc_block(filters, input_layer):
    """Multi-scale空洞卷积融合模块(MCDC) - Multi-scale空洞 convolution fusion module"""
    # Branch 1
    branch1 = conv_bn_relu(filters, (3, 3, 3), 1, input_layer, dilation_rate=1)
    branch1 = conv_bn_relu(filters, (1, 1, 1), 1, branch1)
    
    # Branch 2: 1->2->3
    branch2 = conv_bn_relu(filters, (3, 3, 3), 1, branch1, dilation_rate=2)
    branch2 = conv_bn_relu(filters, (3, 3, 3), 1, branch2, dilation_rate=3)
    branch2 = conv_bn_relu(filters, (1, 1, 1), 1, branch2)
    
    # Branch 3: 1->2->5
    branch3 = conv_bn_relu(filters, (3, 3, 3), 1, branch1, dilation_rate=2)
    branch3 = conv_bn_relu(filters, (3, 3, 3), 1, branch3, dilation_rate=5)
    branch3 = conv_bn_relu(filters, (1, 1, 1), 1, branch3)
    
    # Feature fusion
    fused = Concatenate()([branch1, branch2, branch3])
    fused = conv_bn_relu(filters * 2, (1, 1, 1), 2, fused)
    return conv_bn_relu(filters * 2, (1, 1, 1), 2, fused)


def residual_block(input_x, filters, downsample=False):
    """Residual block with optional downsampling"""
    # Main path
    x = conv_bn_relu(filters, (1, 1, 1), 2 if downsample else 1, input_x)
    x = conv_bn_relu(filters, (3, 3, 3), 1, x)
    x = conv_bn_relu(filters * 4, (1, 1, 1), 1, x)
    
    # Skip connection path
    if downsample:
        input_x = conv_bn_relu(filters * 4, (1, 1, 1), 2, input_x)
    
    return Add()([x, input_x])


def build_3d_resnet50_mcdc_cbam(input_shape=(23, 23, 63, 1), num_classes=17):
    """Build 3D ResNet50 combined with MCDC and CBAM model"""
    input_layer = Input(shape=input_shape, name='input_layer')
    
    # Initial convolution and pooling
    x = conv_bn_relu(64, (7, 7, 7), 1, input_layer)
    x = MaxPooling3D((3, 3, 3), strides=2, padding='same', name='initial_maxpool')(x)
    
    # Residual stages
    stage_outputs = []
    filters = 64
    num_residuals = [3, 4, 6, 3]
    
    for i, num_res in enumerate(num_residuals):
        for j in range(num_res):
            x = residual_block(x, filters, downsample=(j == 0))
        stage_outputs.append(x)
        filters *= 2
    
    # Extract stage outputs
    stage1, stage2, stage3, stage4 = stage_outputs
    
    # Shallow feature processing
    bottleneck1 = mcdc_block(64, stage1)
    bottleneck2 = mcdc_block(128, stage2)
    
    # Shallow feature fusion
    bottleneck1 = conv_bn_relu(256, (1, 1, 1), 1, bottleneck1)
    bottleneck2_up = UpSampling3D(size=(2, 2, 2))(bottleneck2)
    shallow_feature = Multiply()([bottleneck1, bottleneck2_up])
    
    # Deep feature processing
    bottleneck3 = mcdc_block(128, stage3)
    bottleneck2_resample = conv_bn_relu(256, (1, 1, 1), 1, bottleneck2)
    bottleneck3_fused = Multiply()([bottleneck2_resample, bottleneck3])
    bottleneck3_at = cbam_block(bottleneck3_fused)
    
    bottleneck4 = mcdc_block(256, stage4)
    bottleneck3_resample = conv_bn_relu(512, (1, 1, 1), 1, bottleneck3)
    bottleneck4_fused = Multiply()([bottleneck3_resample, bottleneck4])
    bottleneck4_at = cbam_block(bottleneck4_fused)
    
    # Deep feature fusion
    bottleneck3_at_resample = conv_bn_relu(512, (1, 1, 1), 1, bottleneck3_at)
    deep_feature = Multiply()([bottleneck3_at_resample, bottleneck4_at])
    
    # Feature matching and final fusion
    shallow_feature = conv_bn_relu(512, (1, 1, 1), 2, shallow_feature)
    stage4_output = conv_bn_relu(512, (1, 1, 1), 1, stage4)
    
    final_fused = Concatenate()([stage4_output, shallow_feature, deep_feature])
    final_fused = conv_bn_relu(256, (1, 1, 1), 1, final_fused)
    
    # Classification head
    x = GlobalAveragePooling3D()(final_fused)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.4)(x)
    output_layer = Dense(num_classes, activation='softmax', name='output_layer')(x)
    
    # Build model
    model = Model(inputs=[input_layer], outputs=[output_layer])
    return model


In [ ]:
# Load and preprocess data
data_path = os.path.join(os.getcwd(), 'data')
HSI_PU = sio.loadmat(os.path.join(data_path, 'PaviaU.mat'))['paviaU']
print(f"Original HSI shape: {HSI_PU.shape}")
HSI_PU = np.transpose(HSI_PU, (1, 0, 2))
GT_PU = sio.loadmat(os.path.join(data_path, 'PaviaU_gt.mat'))['paviaU_gt']
GT_PU = np.transpose(GT_PU, (1, 0))
print(f"After transpose - HSI: {HSI_PU.shape}, GT: {GT_PU.shape}")
    
# Apply dimensionality reduction
HSI_PU = DL_Method(HSI_PU, numComponents=75)
    
# Extract cubes
HSI_PU_cube, GT_PU_cube = HSICubes(HSI_PU, GT_PU, WinSize=13)
print(f"Extracted cubes - HSI: {HSI_PU_cube.shape}, GT: {GT_PU_cube.shape}")

In [6]:
# Alternatively load pre-saved data
save_dir = os.path.join(os.getcwd(), 'data')
HSI_cube = np.load(os.path.join(save_dir, 'HSI_cube.npy'))
GT_cube = np.load(os.path.join(save_dir, 'GT_cube.npy'))

print("数据加载完成")
print(HSI_cube.shape, GT_cube.shape)

数据加载完成
(37298, 23, 23, 63) (37298,)


In [7]:
# Split into training, validation, and test sets
Tr, Te, TrC, TeC = SplitTr_Te(HSI_cube, GT_cube, 0.8)
Te, Tv, TeC, TvC = SplitTr_Te(Te, TeC, 0.75)
print(f"Dataset splits - Train: {Tr.shape}, Test: {Te.shape}, Validation: {Tv.shape}")
    
# Reshape data for 3D CNN and one-hot encode labels
Tr = Tr.reshape(-1, 23, 23, 63, 1)
TrC = to_categorical(TrC)
Te = Te.reshape(-1, 23, 23, 63, 1) 
TeC = to_categorical(TeC)
Tv = Tv.reshape(-1, 23, 23, 63, 1) 
TvC = to_categorical(TvC)
print(f"After reshaping - Train: {Tr.shape}, Train labels: {TrC.shape}")
print(f"Test: {Te.shape}, Test labels: {TeC.shape}")
print(f"Validation: {Tv.shape}, Validation labels: {TvC.shape}")

Dataset splits - Train: (7459, 23, 23, 63), Test: (7459, 23, 23, 63), Validation: (22380, 23, 23, 63)
After reshaping - Train: (7459, 23, 23, 63, 1), Train labels: (7459, 17)
Test: (7459, 23, 23, 63, 1), Test labels: (7459, 17)
Validation: (22380, 23, 23, 63, 1), Validation labels: (22380, 17)


In [8]:
# Balance training dataset
Tr_balanced, TrC_balanced = balance_dataset(Tr, TrC, target=1500)
    
# Verify class distribution
class_counts = np.sum(TrC_balanced, axis=0)
print("Class distribution after balancing:")
for i in range(17):
    print(f"Class {i}: {int(class_counts[i])} samples")

Original class distribution: [ 685  859 1030  189  544    8  777   83  635   26   14  709  155   83
  155  802  705]
Augmenting class 0: from 685 to 1500
Augmenting class 1: from 859 to 1500
Augmenting class 2: from 1030 to 1500
Augmenting class 3: from 189 to 1500
Augmenting class 4: from 544 to 1500
Augmenting class 5: from 8 to 1500
Augmenting class 6: from 777 to 1500
Augmenting class 7: from 83 to 1500
Augmenting class 8: from 635 to 1500
Augmenting class 9: from 26 to 1500
Augmenting class 10: from 14 to 1500
Augmenting class 11: from 709 to 1500
Augmenting class 12: from 155 to 1500
Augmenting class 13: from 83 to 1500
Augmenting class 14: from 155 to 1500
Augmenting class 15: from 802 to 1500
Augmenting class 16: from 705 to 1500
Balanced class distribution: [1500 1500 1500 1500 1500 1500 1500 1500 1500 1500 1500 1500 1500 1500
 1500 1500 1500]
Class distribution after balancing:
Class 0: 1500 samples
Class 1: 1500 samples
Class 2: 1500 samples
Class 3: 1500 samples
Class 4: 15

In [11]:
# Clean up memory
gc.collect()
    
# Build model
model = build_3d_resnet50_mcdc_cbam(
    input_shape=(23, 23, 63, 1),
    num_classes=17
)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_layer (InputLayer)       [(None, 23, 23, 63,  0           []                               
                                 1)]                                                              
                                                                                                  
 conv3d_204 (Conv3D)            (None, 23, 23, 63,   22016       ['input_layer[0][0]']            
                                64)                                                               
                                                                                                  
 batch_normalization_200 (Batch  (None, 23, 23, 63,   256        ['conv3d_204[0][0]']             
 Normalization)                 64)                                                         

In [13]:
# Define callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-5
    )
]
    
# Compile model
optimizer = Adam(learning_rate=0.001, decay=1e-06)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
# Train model
start_time = time.time()
history = model.fit(
    x=Tr_balanced, 
    y=TrC_balanced, 
    batch_size=128, 
    epochs=100, 
    validation_data=(Te, TeC),
    callbacks=callbacks
)
training_time = time.time() - start_time
print(f"Training time: {training_time:.2f} seconds")

Epoch 1/100
200/200 [==============================] - 83s 361ms/step - loss: 1.8314 - accuracy: 0.4040 - val_loss: 1.6161 - val_accuracy: 0.4672 - lr: 0.0010
Epoch 2/100
200/200 [==============================] - 66s 330ms/step - loss: 0.7479 - accuracy: 0.7612 - val_loss: 0.8130 - val_accuracy: 0.7378 - lr: 0.0010
Epoch 3/100
200/200 [==============================] - 66s 332ms/step - loss: 0.4721 - accuracy: 0.8549 - val_loss: 0.3951 - val_accuracy: 0.8862 - lr: 0.0010
Epoch 4/100
200/200 [==============================] - 66s 330ms/step - loss: 0.3566 - accuracy: 0.8896 - val_loss: 0.3277 - val_accuracy: 0.8914 - lr: 0.0010
Epoch 5/100
200/200 [==============================] - 66s 329ms/step - loss: 0.2874 - accuracy: 0.9120 - val_loss: 0.2669 - val_accuracy: 0.9192 - lr: 0.0010
Epoch 6/100
200/200 [==============================] - 66s 329ms/step - loss: 0.2132 - accuracy: 0.9342 - val_loss: 0.2031 - val_accuracy: 0.9406 - lr: 0.0010
Epoch 7/100
200/200 [=========================

In [ ]:
# Evaluate model
predictions = model.predict(Tv)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(TvC, axis=1)
    
# Calculate and save confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)
cm_df = pd.DataFrame(cm)
cm_df.to_csv(os.path.join(save_dir, 'model_confusion_T1500.csv'), index=False, header=True)
    
# Calculate and save precision matrix
precision_matrix = cm.astype('float') / cm.sum(axis=0, keepdims=True)
pm_df = pd.DataFrame(precision_matrix)
pm_df.to_csv(os.path.join(save_dir, 'model_precision_T1500.csv'), index=False, header=True)
    
# Calculate evaluation metrics
oa = accuracy_score(true_classes, predicted_classes)
print(f"Overall Accuracy (OA): {oa:.4f}")
    
aa = np.mean(np.diag(cm) / np.sum(cm, axis=1))
print(f"Average Accuracy (AA): {aa:.4f}")
    
recall = recall_score(true_classes, predicted_classes, average=None)
print("Class-wise Recall:")
for class_idx, r in enumerate(recall):
    print(f"Class {class_idx}: {r:.4f}")
    
f1 = f1_score(true_classes, predicted_classes, average=None)
print("Class-wise F1 Score:")
for class_idx, f in enumerate(f1):
    print(f"Class {class_idx}: {f:.4f}")
    
kappa = cohen_kappa_score(true_classes, predicted_classes)
print(f"Kappa Coefficient: {kappa:.4f}")